# STADVDB MCO1: Query Processing in a Data Warehouse
## Research Question: The Impact of Language, Region, and Cultural Adaptation on International Film Reception

**STADVDB_S19 Group 10:**  
- Perol, Francine Nicole T.
- Pineda, Dencel Angelo D.
- Sy, Vaughn Marick A.
- Valdez, Pulvert Gerald M.

In [3]:
from sqlalchemy import create_engine, text

engine = create_engine(
    "postgresql+psycopg2://",
    connect_args={
        "host": "localhost",
        "port": 5432,
        "dbname": "mco1_imdb",
        "user": "postgres",
        "password": "EleanorKat17;"
    }
)

with engine.connect() as conn:
    print("✅ Connected via SQLAlchemy:", conn.execute(text("SELECT version();")).fetchone())


✅ Connected via SQLAlchemy: ('PostgreSQL 18.0 on x86_64-windows, compiled by msvc-19.44.35217, 64-bit',)


In [ ]:
# Test: fetch first 5 rows from your fact table
df = pd.read_sql("SELECT * FROM fact_film_version LIMIT 5;", engine)
df